#### **DC/DC converter** :

The studied DC/DC converter is composed by one film capacitor, two IGBT transistors and one inductance.


##### **Losses evaluation**

In [98]:
import numpy as np

def converter():

    #Data
    E = 250                                 #bus voltage
    Il = 188                                #motor voltage
    Ul = 140                                #motor current
    

    #Design assumptions
    dE = 0.02*E                             #Max ripple voltage (2%)
    dI = 16.94/100*Il                       #Max ripple current (16.94%)

   
   
    #Optimization variables
    f = 9098                                #switching frequency
    
    eD = 10**(-1)                            #airgap/diameter INDUCTOR
    J = 10.5*10**6                           #current density [A/mm^2] INDUCTOR

    DH = 0.63                                #aspect ratio CAPACITOR
    kc = 1.5                                 #oversizing coefficient CAPACITOR

    kIGBT = 3.3                              #oversizing coefficient IGBT
    Wh = 105                                 #width heatsink [mm] IGBT



    alpha = Ul/E
    T = 1/f

    L = (1-alpha)*alpha*E*T/dI
    Il_max = Il+dI/2
    Il_RMS = Il*np.sqrt(1+(dI/Il)**2/12)

    C = alpha*T*(1-alpha)*Il/dE
    Ic_RMS = np.sqrt(alpha*(1-alpha))*Il*np.sqrt(1+1/(12*(1-alpha))*(dI/Il)**2)

    I_RMS = np.sqrt(alpha)*Il*np.sqrt(1+1/12*(dI/Il)**2)
    I_mean = alpha*Il

    Id_RMS = np.sqrt(1-alpha)*Il*np.sqrt(1+1/12*(dI/Il)**2)
    Id_mean = (1-alpha)*Il

    dI2 = dI/2

    #INDUCTOR
    #projectual hypotesis
    J_fil = 10.53*10**6
    eD_n = 0.0999999
    mu0 = 1.25664*10**(-6)
    wind_coeff = 0.33
    Bmax = 0.4

    Em = 0.5*L*Il_max**2            #magnetic energy
    Pi0 = 3.86*eD_n**(0.344-0.226*np.log10(eD_n)-0.0355*np.log10(eD_n)**2)
    Dl = (2*Pi0*Em/(mu0*J_fil**2*(0.078+0.195*eD_n)**2*wind_coeff**2))**(1/5)     #inductance diameter
    Rl = Pi0/(mu0*Dl)                #riluctance
    N = round(np.sqrt(L*Rl), 0) +1    #number of turns
    A_fil = Il_max/J_fil            #fil surface
    A_bob = A_fil*N/wind_coeff
    B = N*Il_max/Rl/(Dl**2*np.pi*(0.82/2)**2)
    dB = N*dI2/Rl/(Dl**2*np.pi*(0.82/2)**2)
    e = eD_n*Dl

    M1 = wind_coeff*8960*0.153*0.142**3        #0.142 is the dimension A as in picture 
    M2 = (1-wind_coeff)*1200*0.153*0.142**3
    M3 = 0.487*7870*0.1472**3
    M_ind = M1+M2+M3    


    #CAPACITOR
    lam = 0.3
    C_n = C*kc

    C_ref = 1000*10**(-6)
    D_ref = 100*10**(-3)
    H_ref = 155*10**(-3)

    Dc = (C_n/C_ref*(DH/(D_ref/H_ref)))**(1/3)*D_ref        #scaling law
    Hc = Dc/DH
    Rc_ref = 3.2*10**(-3)
    Rc = (Dc/D_ref)**(-2)*Rc_ref

    #losses
    tandel = 2*10**(-4)
    Pj_c = Ic_RMS**2*Rc
    P_di = 0.5*C_n*dE**2*2*np.pi*f*tandel

    M_ref = 1.5
    M_c = M_ref*(Dc/D_ref)**2*(Hc/H_ref)


    #CHOPPER

    #IGBT
    I_n = I_mean*kIGBT
    I_ref = 80
    V_ref = 600
    comm_ref = 8.2*10**(-3)
    R0_ref = 20*10**(-3)
    V0 = 1              #voltage drop
    R0 = R0_ref*(I_n/I_ref)**(-1)           #dynamic resistance
    J_comm = comm_ref*(I_n/I_ref)*(E/V_ref)*Il/I_n
    V_max = 900

    #DIODE
    Id_ref = 41
    Vd_ref = 600
    R0d_ref = 15*10**(-3)
    commd_ref = 17.2*10**(-3)
    Id_n = Id_ref*(I_n/I_ref)
    R0d = R0d_ref*(I_n/I_ref)**(-1)
    Jd_comm = commd_ref*(I_n/I_ref)*(E/V_ref)*Il/Id_n

    Lh = 0.26
    Wh = 0.11
    ef = Wh*0.025
    N_fin = round((Wh-ef)/(2*ef), 0)
    Md = 2700*Lh*(0.36*Wh**2+0.02*Wh**2*N_fin)

    J_IGBT = V0*I_mean+R0*I_RMS**2+J_comm*f
    J_diode = V0*Id_mean+R0d*Id_RMS**2+Jd_comm*f

    print('Inductor mass', M_ind)
    print('Capacitor mass', M_c)
    print('Diode mass', Md)
    print('IGBT losses', J_IGBT)
    print('diode losses', J_diode)
   
    return 



In [99]:
converter()


Inductor mass 13.871930722200318
Capacitor mass 2.2912112552209267
Diode mass 6.2857080000000005
IGBT losses 269.6988498759596
diode losses 435.5392889331474
